In [ ]:
import hoomd

import hoomd.pair_plugin.pair as ppair

from monk import prep, pair

import ex_render

import numpy as np

In [ ]:
N = 512
D = 2
phi_initial = 0.6
phi_final = 0.6
L = prep.len_from_phi(N, phi_initial, dim=D)
L_final = prep.len_from_phi(N, phi_final, dim=D)

In [ ]:
device = hoomd.device.auto_select()
sim = hoomd.Simulation(device, seed=1234)

sim.create_state_from_gsd("new_test_2d.gsd")

In [ ]:
class RewrapStrain(hoomd.custom.Action):

    def act(self, timestep):
        box = self._state.box
        xy = box.xy
        if xy > 0.5:
            xy -= 1.0
        elif xy < -0.5:
            xy += 1.0
        box.xy = xy
        if not box.is2D:
            yz = box.yz
            if yz > 0.5:
                yz -= 1.0
            elif yz < -0.5:
                yz += 1.0
            box.yz = yz
            xz = box.xz
            if xz > 0.5:
                xz -= 1.0
            elif xz < -0.5:
                xz += 1.0
            box.xz = xz
        self._state.set_box(box)

In [ ]:

k = 10.0

# P = [0, 0.2, 0, -.05, 0, 0]

integrator = hoomd.md.Integrator(dt=0.01, integrate_rotational_dof=True)
# nvt = hoomd.md.methods.NPH(hoomd.filter.All(), 0.2, 1.0, couple="none", box_dof=(False, True, False, False, False, False), gamma=0.1)
nvt = hoomd.md.methods.NPH(hoomd.filter.All(), 3*[0.2] + 3*[0.0], 1.0, couple="none", box_dof=(False, True, False, False, False, False), gamma=0.1)

tree = hoomd.md.nlist.Tree(0.2)
flj = ppair.HarmHPF(tree, default_r_cut=1.0, mode="none", mus=1.0, ks=10.0, mur=1.0, kr=10.0)
flj.params[('A', 'A')] = dict(k=10.0, rcut=1.0)
flj.r_cut[('A', 'A')] = 1.0
flj.params[('A', 'B')] = dict(k=10.0, rcut=1.2)
flj.r_cut[('A', 'B')] = 1.2
flj.params[('B', 'B')] = dict(k=10.0, rcut=1.4)
flj.r_cut[('B', 'B')] = 1.4

integrator.forces = [flj]
integrator.methods = [nvt]

sim.operations.integrator = integrator

writer = hoomd.write.GSD(trigger=hoomd.trigger.Periodic(100), filename="ver2_test_constant_stress.gsd", mode="wb", filter=hoomd.filter.All(), dynamic=["property", "momentum"])
sim.operations.writers.clear()
sim.operations.writers.append(writer)

custom_action = RewrapStrain()
fix_box = hoomd.update.CustomUpdater(hoomd.trigger.Periodic(10), custom_action)

sim.operations.updaters.clear()
sim.operations.updaters.append(fix_box)

In [ ]:
sim.run(0)

In [ ]:
flj._cpp_obj.gamma = 1.0

In [ ]:
sim.run(60_000)
ex_render.render_disk_frame(frame=sim.state.get_snapshot())

In [ ]:

class MyConstantShear(hoomd.custom.Action):
    """Apply a constant shear rate to the simulation box.
    
    Arguments
    ---------
    - gamma: float - the shear rate (in units of box ratio per time step)
    - timestep: int - the initial reference timestep
    - pair: str - the dimension pair to apply the shear to"""
    def __init__(self, gamma: float, timestep: int, pair: str = "xz"):
        super().__init__()
        self._gamma = gamma
        self._last_timestep = timestep
        match pair:
            case "xy":
                self._pair_mode = 0
            case "yz":
                self._pair_mode = 1
            case "xz":
                self._pair_mode = 2

    def act(self, timestep):
        time_diff = timestep - self._last_timestep
        self._last_timestep = timestep
        box = self._state.box
        snap = self._state.get_snapshot()
        match self._pair_mode:
            case 0: # xy
                xy = box.xy + self._gamma * time_diff
                if xy > 0.5:
                    xy -= 1.0
                elif xy < -0.5:
                    xy += 1.0
                box.xy = xy
                snap.particles.position[:, 0] += snap.particles.position[:, 1] * self._gamma * time_diff * box.Lx / box.Ly
            case 1: # yz
                yz = box.yz + self._gamma * time_diff
                if yz > 0.5:
                    yz -= 1.0
                elif yz < -0.5:
                    yz += 1.0
                box.yz = yz
            case 2: # xz
                xz = box.xz + self._gamma * time_diff
                if xz > 0.5:
                    xz -= 1.0
                elif xz < -0.5:
                    xz += 1.0
                box.xz = xz
        self._state.set_snapshot(snap)
        self._state.set_box(box)

In [ ]:
custom_action = MyConstantShear(1e-4, sim.timestep, pair="xy")
box_resize = hoomd.update.CustomUpdater(hoomd.trigger.Periodic(10), custom_action)

sim.operations.updaters.clear()
sim.operations.updaters.append(box_resize)

In [ ]:
len(sim.operations.updaters)

In [ ]:
sim.run(40_000)